# Prioritized Experience Replay (PER)

## Table of Contents
- [1 - Motivation](#1)
- [2 - Prioritized Experience Replay](#2)


Full paper: [Prioritized experience replay (2016)](https://arxiv.org/pdf/1511.05952.pdf)

<a name='1'></a>
# 1 - Motivation

Online Reinforcement Learning agents incrementally update their parameters (of the policy, value function or model) while they observe a stream of experience. In their simplest form, they discard incoming data immediately, after a single update. Two issues with this are (a) strongly correlated updates that break the i.i.d. assumption of many popular stochastic gradient-based algorithms, and (b) the rapid forgetting of possibly rare experiences that would be useful later on. 

Experience Replay addresses both of these issues: with experience stored in a replay memory, it becomes possible to break the temporal correlations by mixing more and less recent experience for the updates, and rare experience will be used for more than just a single update. In general, experience replay can reduce the amount of experience required to learn, and replace it with more computation and more memory - which are often cheaper resources then the RL agent's interactions with its environment.

Which the original experience replay technique we have a fixed length buffer, where we store each timesteps experience tuple so that we can randomly sample a batch of tuples to train the agent at the same time. But there is one main improvement that we can make and and this is got to do with how we sample experiences to train from the buffer. The key idea behind prioritized experience replay is to prioritize experiences, that the agent can learn more from than from others. For example an action that causes the environement to terminate. The aim is to make the most effective use of the replay memory for learning, assuming that its contents are outside of our control.

Experience replay liberates online learning agents from processing transitions in the exact order they are experienced. Prioritized replay further liberates agents from considering transitions with the same frequency that they are experienced.

<a name='2'></a>
# 2 - Prioritized Experience Replay

**Prioritizing with TD-Error**

A central component of prioritized experience replay is the criterion by which the importance of each transistion is measured. The transition's TD error $\delta$ is a reasonable proxy of the the amount the RL agent can learn from a transition in its current state (expected learning progress). This criterion is particulararly suitable for incremental, online RL algorithms, such as SARSA or Q-Learning. These algorithmns already compute the TD error and update their parameters in proportion to $\delta$, which is the difference between the predicted q-value for the state action pair and the q-target value calculated from the bellman equation:

$$\delta = q(s, a) - q_{target}$$

There is just one small issue here, that is when the error turns out to be zero. We don't want our priority weight to be zero as this experience might be useful later on. So we add a small offset-value $\epsilon$ to each priority to prevent the edge-case of transitions not being revisited once their error is zero. Now we can actually set our priority value for each tuple to be directly proportional to the absolut value of this calculated error:

$$P(i) = \vert \delta_{i} \vert + \epsilon$$

When we sample experiences we can convert these priority weights of each tuple to a probability of chosing that tuple for the current batch by deviding each priority value by the total sum of all priority weights. However, greedy TD-error prioritization has several issues. It focuses on a small subset of experience and errors shrink slowly, especially when using function approximation, meaning that the initially high error transitions get replay frequently. This lack of diversity makes the system prone to overfitting. To overcome these issues, each priority is raised to a power of a priority scaling constant $\alpha$. This leads to a stochastic sampling method, where $\alpha$ equals one results in fully priority sampling and $\alpha$ equals zero results in pure random sampling of normal experience replay:

$$P_{r}(i) = \frac{P_{i}^\alpha} {\sum_{k}{P_{k}^\alpha}}$$

Now we have taken care of the sampling of experience according to the priorities as well as setting the priorities of each sampled batch of experience tuples. But there is one more adjustment to make.

**Importance sampling weight**

Prioritized replay introduces bias towards those experiences, that have a higher priority, as we are now sampling experience non uniformly. To understand why, let's look at how we update the Q-network for each state-acton pair. For each optimization problem we first need to define a loss function that we want to minimize, which in our case is the squared TD-error:

$$loss = \vert \delta^2 \vert$$

Then for each layer in the Q-network we update the weights and biases, which are represented through $\theta$, towards the negativ gradient of the loss function and weight that by a learning rate $\alpha$:

$$\theta = \theta - \alpha \frac{dloss} {d\theta}$$

Ideally we do that for each state-action pair the environment gives us. With experience replay we are approximating the environment with the replay buffer and accelerating the training by sampling a batch of experience to train on. And because we are sampling randomly, the original distribution of experiences that the environment produces is preserved. However, once we introduce sampling priorities to the replay buffer, this changes the distribution if favor of the higher prioritized experiences, making the network anticipate these experiences more and overfit them. 

We can correct this bias by using importance sampling weights that fully compensates for the non-uniform probabilities $P(i)$ if $\beta=1$:

$$w_{i} = \left(\frac{1} N \cdot \frac{1} {P_{r}(i)}\right)^\beta$$

The value of the weight factor $w_{i}$ is one devided by the total replay buffer size, times one devided by the probability. This will reduce the step size for higher probabilitiy experiences and therefore avoid excessiv update steps from the increased frequency of training on these experiences. Since the unbiased nature ofthe updates is most important near convergence at the end of training, as the processs is highly non-stationary anyay, we can introduce a schedule on the exponent $\beta$ that reaches 1 only at the end of learning. In practice, we linnearly anneal $\beta$ rom its initial value $\beta_{0}$ to 1. For stability reasons, we can also normalize weights by $\frac{1} {\max_{i} w_{i}}$, so that they only scale the update downwoards.

These importance sampling weights can be folded into into the Q-learning update by using $w_{i} \delta_{i}$ instead of $\delta_{i}$:

$$\theta = \theta - \alpha \cdot \frac{dloss} {d\theta} w_{i}$$